In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def load_dataset(directories):
    dataset = []
    for directory in directories:
        for filename in os.listdir(directory):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image = cv2.imread(os.path.join(directory, filename))
                if image is not None:
                    dataset.append((filename, image))
    return dataset

def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

def compare_descriptors(descriptors1, descriptors2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return len(matches)

def find_similar_images(input_image_path, directories):
    input_image = cv2.imread(input_image_path)
    if input_image is None:
        print("Erreur : l'image n'a pas été chargée correctement.")
        return []

    _, input_descriptors = extract_features(input_image)
    dataset = load_dataset(directories)

    scores = []
    for filename, image in dataset:
        _, descriptors = extract_features(image)
        score = compare_descriptors(input_descriptors, descriptors)
        scores.append((filename, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

def display_images(image_paths, directories):
    plt.figure(figsize=(10, 10))
    for i, (filename, score) in enumerate(image_paths[:5]):
        for directory in directories:
            image_path = os.path.join(directory, filename)
            if os.path.exists(image_path):
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                plt.subplot(1, 5, i + 1)
                plt.imshow(image)
                plt.title(f"Score: {score}")
                plt.axis('off')
                break
    plt.show()

# Exemple d'utilisation
input_image_path = r"C:\Users\SONA8\Desktop\projet_eval\chat\chat2.jpg"
directories = [r"C:\Users\SONA8\Desktop\projet_eval\chat", r"C:\Users\SONA8\Desktop\projet_eval\lampe", r"C:\Users\SONA8\Desktop\projet_eval\train"]

similar_images = find_similar_images(input_image_path, directories)
print("Les images les plus similaires sont :")
for filename, score in similar_images:
    print(f"{filename}: {score}")

# Afficher les images les plus similaires
display_images(similar_images, directories)
